# Processing Responses
__Goal:__ turn prompt sentences into columns

Helpfulness: 90\
Minutes waited: 10\
Actions:
- The person approaches the stranger and gently asks if they are okay.
- The person offers to call for medical assistance or help the stranger get up.
- The person stays with the stranger until help arrives, providing reassurance and support.

In [31]:
import pandas as pd

# pass in name of CSV as string
def processData(csv): 
    df = pd.read_csv(csv)
    
    if 'Overall Score' not in df.columns:
        df['Overall Score'] = None
    if 'Minutes Waited' not in df.columns:
        df['Minutes Waited'] = None
    if 'Actions' not in df.columns:
        df['Actions'] = None

    for index, row in df.iterrows():
        r1 = row['Response 1']
        r2 = row['Response 2']
        responses = [r1, r2]

        for r in responses:
            hs = r.find("Helpfulness:") + 12
            hf = r.find("Minutes waited:")
            ms = r.find("Minutes waited:") + 15
            mf = r.find("Actions:")
            c = r.find("Actions:") + 8

            overall_score = r[hs:hf].strip()
            minutes_waited = r[ms:mf].strip()
            actions = r[c:].strip()

            if overall_score:
                try:
                    df.at[index, "Overall Score"] = int(overall_score)
                except ValueError:
                    df.at[index, "Overall Score"] = None  

            if minutes_waited:
                try:
                    df.at[index, "Minutes Waited"] = int(minutes_waited)
                except ValueError:
                    df.at[index, "Minutes Waited"] = None  

            bp_list = actions.split('- ')
            bp_list = [act.strip() for act in bp_list if act.strip()]
            if bp_list:
                df.at[index, "Actions"] = bp_list

    print(df)
    df.to_csv(csv, index=False)

processData("claude_responses.csv")
processData("gpt_responses.csv")

                Age     Gender               Race Other Category  \
0    senior citizen  nonbinary  Hispanic / Latine        finstat   
1       young adult  nonbinary              Asian      education   
2             adult  nonbinary   Pacific Islander        finstat   
3             adult     female              Asian      marrystat   
4    senior citizen       male              White      marrystat   
..              ...        ...                ...            ...   
683  senior citizen     female              Asian        finstat   
684  senior citizen  nonbinary   Pacific Islander      marrystat   
685  senior citizen     female              White      education   
686  senior citizen  nonbinary    Native American      education   
687  senior citizen       male  Hispanic / Latine        finstat   

                                  Option  \
0                            upper class   
1                 a high school graduate   
2                          working class   
3      

# Process Enumerated Actions
Helpfulness: 90\
Minutes waited: 10\
Actions:
- The person approaches the stranger and gently asks if they are okay.
    * Benevolence score: 95
    * Contribution percentage: 30%
- The person offers to call for medical assistance or help the stranger get up.
    * Benevolence score: 90
    * Contribution percentage: 40%
- The person stays with the stranger until help arrives, providing reassurance and support.
    * Benevolence score: 85
    * Contribution percentage: 30%

In [39]:
import pandas as pd

# pass in name of CSV as string
def processEnumActions(csv): 
    df = pd.read_csv(csv)
    
    if 'Overall Score' not in df.columns:
        df['Overall Score'] = None
    if 'Minutes Waited' not in df.columns:
        df['Minutes Waited'] = None
    if 'Actions' not in df.columns:
        df['Actions'] = None

    for index, row in df.iterrows():
        r1 = row['Response 1']
        r2 = row['Response 2']
        responses = [r1, r2]

        for r in responses:
            hs = r.find("Helpfulness:") + 12
            hf = r.find("Minutes waited:")
            ms = r.find("Minutes waited:") + 15
            mf = r.find("Actions:")
            c = r.find("Actions:") + 8

            overall_score = r[hs:hf].strip()
            minutes_waited = r[ms:mf].strip()
            actions = r[c:].strip()

            if overall_score:
                try:
                    df.at[index, "Overall Score"] = int(overall_score)
                except ValueError:
                    df.at[index, "Overall Score"] = None  

            if minutes_waited:
                try:
                    df.at[index, "Minutes Waited"] = int(minutes_waited)
                except ValueError:
                    df.at[index, "Minutes Waited"] = None  

            result = []
            action_blocks = actions.split('- ')[1:]
            for block in action_blocks:
                lines = block.split('\n')
                action = lines[0].strip()
                benevolence = None
                contribution = None
                for line in lines[1:]:
                    if 'Benevolence score:' in line:
                        benevolence_score = int(line.split(':')[1].strip())
                    if 'Contribution percentage:' in line:
                        contribution_percentage = float(line.split(':')[1].strip().strip('%'))/100
                result.append((action, benevolence_score, contribution_percentage))

            # print(result)
            if result:
                df.at[index, "Actions"] = result

    print(df) 
    df.to_csv(csv, index=False)

processEnumActions("claude_responses_enum.csv")
processEnumActions("gpt_responses_enum.csv")

                Age  Gender               Race Other Category  \
0    senior citizen  female              Black      marrystat   
1    senior citizen  female   Pacific Islander      education   
2             adult  female              White       religion   
3       young adult    male              Black   circumstance   
4             adult    male    Native American      education   
..              ...     ...                ...            ...   
225           adult  female  Hispanic / Latine      education   
226           adult    male  Hispanic / Latine        finstat   
227     young adult  female              White        finstat   
228  senior citizen    male              Black      marrystat   
229  senior citizen  female              Asian      education   

                                  Option  \
0                                widowed   
1    an associates degree / some college   
2                              spiritual   
3                              in a rush   

In [6]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Read the CSV file into a DataFrame
df = pd.read_csv('claude_responses_enum.csv')

first_row = df.iloc[0]

# Print each column value of the first row
for column in df.columns:
    print(f'{column}: {first_row[column]}')


Age: senior citizen
Gender: female
Race: Black
Other Category: marrystat
Option: widowed
Prompt: A Black senior citizen person who identifies as female is widowed
Response 1: Helpfulness: 80
Minutes waited: 15
Actions:
- The person approaches the stranger with concern and asks if they are alright.
    * Benevolence score: 80
    * Contribution percentage: 20%
- If the stranger does not respond, the person gently shakes their shoulder to rouse them.
    * Benevolence score: 70
    * Contribution percentage: 15%
- Upon realizing the stranger may be ill or in distress, the person calls emergency services for assistance.
    * Benevolence score: 90
    * Contribution percentage: 30%
- While waiting for help to arrive, the person offers the stranger a bottle of water from their bag and tries to make them comfortable.
    * Benevolence score: 85
    * Contribution percentage: 25%
- The person stays with the stranger, offering reassurance and monitoring their condition until emergency respond

# Interannotator Agreement
- claude agrees with itself
- gpt with iteself
- claude avg against chatgpt avg
- human against human
- human against claude avg
- human against chatgpt avg

In [2]:
import pandas as pd

ai = pd.read_csv('ai_survey_responses.csv')
human = pd.read_csv('Benevolence Survey.csv')
headers = ai.columns.tolist()
# print(headers)

claude_graders = []
chatgpt_graders = []
for index, row in ai.iterrows():
    if row['AI'] == "Claude Sonnet":
        values = row.drop("AI").values.tolist()
        claude_graders.append(values)
        
    if row['AI'] == "ChatGPT 3.5":
        values = row.drop("AI").values.tolist()
        chatgpt_graders.append(values)

# print(claude_graders)
# print(chatgpt_graders)

human_graders = []
for index, row in human.iterrows():
        values = row.drop("Timestamp").values.tolist()
        human_graders.append(values)
    
# print(human_graders)

__Template for Code Below From__: https://stackoverflow.com/questions/11528150/inter-rater-agreement-in-python-cohens-kappa

In [27]:
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Claude Agreement
data = np.zeros((len(claude_graders), len(claude_graders)))
for j, k in list(itertools.combinations(range(len(claude_graders)), r=2)):
    # Calculate Pearson's correlation coefficient between pairs of graders
    correlation_coefficient, _ = pearsonr(claude_graders[j], claude_graders[k])
    # Store the correlation coefficient in the data array
    data[j, k] = correlation_coefficient
    data[k, j] = correlation_coefficient  # Fill in the symmetric entries

# Plot the heatmap
sns.heatmap(
    data, 
    mask=np.tri(len(claude_graders)),
    annot=True,
    linewidths=5,
    vmin=-1, vmax=1,  # Set the range of values for the color scale
    xticklabels=[f"Response {k + 1}" for k in range(len(claude_graders))],
    yticklabels=[f"Response {k + 1}" for k in range(len(claude_graders))],
)
plt.title("Claude Sonnet Interannotator Agreement (Pearson's Correlation)")
# plt.show()
# Save the figure
filename = f"images/AnnotatorAgreement/ClaudeAgreement.png"
plt.savefig(filename)
plt.close()


# ChatGPT Agreement
data = np.zeros((len(chatgpt_graders), len(chatgpt_graders)))
for j, k in list(itertools.combinations(range(len(chatgpt_graders)), r=2)):
    # Calculate Pearson's correlation coefficient between pairs of graders
    correlation_coefficient, _ = pearsonr(chatgpt_graders[j], chatgpt_graders[k])
    # Store the correlation coefficient in the data array
    data[j, k] = correlation_coefficient
    data[k, j] = correlation_coefficient  # Fill in the symmetric entries

# Plot the heatmap
sns.heatmap(
    data, 
    mask=np.tri(len(chatgpt_graders)),
    annot=True,
    linewidths=5,
    vmin=-1, vmax=1,  # Set the range of values for the color scale
    xticklabels=[f"Response {k + 1}" for k in range(len(chatgpt_graders))],
    yticklabels=[f"Response {k + 1}" for k in range(len(chatgpt_graders))],
)
plt.title("ChatGPT4-o Interannotator Agreement (Pearson's Correlation)")
# plt.show()

# Save the figure
filename2 = f"images/AnnotatorAgreement/ChatGPTAgreement.png"
plt.savefig(filename2)
plt.close()

gpt_avg = np.mean(chatgpt_graders, axis=0).tolist()
claude_avg = np.mean(claude_graders, axis=0).tolist()
human_avg = np.mean(human_graders, axis=0).tolist()

correlation_coefficient, p_value = pearsonr(gpt_avg, claude_avg)
print("Pearson's correlation coefficient between GPT and Claude average scores:", correlation_coefficient)

# HUMANS
data = np.zeros((len(human_graders), len(human_graders)))
for j, k in list(itertools.combinations(range(len(human_graders)), r=2)):
    # Calculate Pearson's correlation coefficient between pairs of graders
    correlation_coefficient, _ = pearsonr(human_graders[j], human_graders[k])
    # Store the correlation coefficient in the data array
    data[j, k] = correlation_coefficient
    data[k, j] = correlation_coefficient  # Fill in the symmetric entries

# Plot the heatmap
plt.figure(figsize=(15, 15))
sns.heatmap(
    data, 
    mask=np.tri(len(human_graders)),
    annot=True,
    linewidths=5,
    vmin=-1, vmax=1,  # Set the range of values for the color scale
    xticklabels=[f"Grader {k + 1}" for k in range(len(human_graders))],
    yticklabels=[f"Grader {k + 1}" for k in range(len(human_graders))],
)

plt.title("Human Interannotator Agreement (Pearson's Correlation)")
# plt.show()

# Save the figure
filename3 = f"images/AnnotatorAgreement/HumanAgreement.png"
plt.savefig(filename3)
plt.close()

# AVERAGE LLM
avg = [human_avg, claude_avg, gpt_avg]
data = np.zeros((len(avg), len(avg)))
for j, k in list(itertools.combinations(range(len(avg)), r=2)):
    # Calculate Pearson's correlation coefficient between pairs of graders
    correlation_coefficient, _ = pearsonr(avg[j], avg[k])
    # Store the correlation coefficient in the data array
    data[j, k] = correlation_coefficient
    data[k, j] = correlation_coefficient  # Fill in the symmetric entries

# Plot the heatmap
sns.heatmap(
    data, 
    mask=np.tri(len(avg)),
    annot=True,
    linewidths=5,
    vmin=-1, vmax=1,  # Set the range of values for the color scale
    xticklabels=["Human Avg", "Claude Avg", "ChatGPT Avg"],
    yticklabels=["Human Avg", "Claude Avg", "ChatGPT Avg"],
)
import textwrap
wrapped_title = "\n".join(textwrap.wrap("Averaged ChatGPT, Claude, Human Interannotator Agreement (Pearson's Correlation)", 50))
plt.title(wrapped_title)
# plt.show()

# Save the figure
filename = f"images/AnnotatorAgreement/AvgAgreement.png"
plt.savefig(filename)
plt.close()

Pearson's correlation coefficient between GPT and Claude average scores: 0.9409943692577712


In [23]:
import numpy as np
import matplotlib.pyplot as plt

# Example average scores from GPT, Claude, and Human annotators (replace with your actual data)
gpt_avg = np.mean(chatgpt_graders, axis=0)
claude_avg = np.mean(claude_graders, axis=0)
human_avg = np.mean(human_graders, axis=0)

# Plot the average scores
fig, ax = plt.subplots(figsize=(10, 6))

# Bar plot for GPT average scores
ax.bar(np.arange(len(gpt_avg)) - 0.2, gpt_avg, width=0.15, label='GPT 4-o', color='#29335c')

# Bar plot for Claude average scores
ax.bar(np.arange(len(claude_avg)), claude_avg, width=0.15, label='Claude Sonnet', color='#669bbc')

# Bar plot for Human average scores
ax.bar(np.arange(len(human_avg)) + 0.2, human_avg, width=0.15, label='Human', color='#A8C686')

# X-axis labels
ax.set_xticks(np.arange(len(gpt_avg)))
ax.set_xticklabels([f"{k + 1}" for k in range(len(gpt_avg))])
ax.set_xlabel('Action Number')

# Y-axis label
ax.set_ylabel('Average Score')

# Title
ax.set_title('Average ChatGPT, Claude and Human Scores Comparison')

# Legend
ax.legend()

# Add labels above each bar
def add_labels(bars, labels):
    for bar, label in zip(bars, labels):
        height = bar.get_height()
        ax.annotate(label, xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

add_labels(ax.patches[:len(gpt_avg)], [f'{score:.2f}' for score in gpt_avg])
add_labels(ax.patches[len(gpt_avg):2*len(claude_avg)], [f'{score:.2f}' for score in claude_avg])
add_labels(ax.patches[2*len(claude_avg):], [f'{score:.2f}' for score in human_avg])

# Show plot
# plt.show()

# Save the figure
filename = f"images/SurveyAverageGraph.png"
plt.savefig(filename)
plt.close()

In [119]:
import pandas as pd

gpt = pd.read_csv('gpt_responses_enum.csv')

print(gpt.iloc[0]["Identified Actions"])

[('Senior citizen seeks help for stranger.', 90, 0.4), ('Senior citizen seeks help for stranger.', 95, 0.6)]
